In [1]:
from dataloaders import data_set,data_dict
import torch
import yaml
import os
import torch.nn as nn

import torch
import torch.optim as optim

from dataloaders import data_set,data_dict
from models.model_builder import model_builder
import pandas as pd

import numpy as np

import ray
from ray import tune, air
from ray.air import session, Checkpoint
from ray.tune.examples.mnist_pytorch import ConvNet, get_data_loaders, test
from ray.tune.schedulers import PopulationBasedTraining

import random
from ray.tune.schedulers.pb2 import PB2

from pbtexperiment import train
from analysis import cal_test_accuracy

/home/janis/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-16 19:05:15,228	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-16 19:05:15,276	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-16 19:05:15,349	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
config_path = os.path.abspath('configs/data.yaml')
model_config = os.path.abspath('configs/model.yaml')
root_path = os.path.abspath("datasets")
to_save_path = os.path.abspath("ISWC2022LearnableFilter/Run_logs")   
freq_save_path = os.path.abspath("ISWC2022LearnableFilter/Freq_data")
window_save_path = os.path.abspath("ISWC2022LearnableFilter/Sliding_window")

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    def __getattr__(self, attr):
        if attr.startswith('__'):
            raise AttributeError("Should not start with __")
        return self.get(attr)
    #__getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

def get_args():
    args = dotdict()   
    
    
    args.training_iterations = 100
    args.restore = False
    args.perturbation_interval = 3
    args.trials = 16
    args.scheduler = "pbt"
    args.storage_path = "/home/janis/PopulationBasedTraining/RayTuneResults/oppo/pbt"
    args.cpu_per_trial = 1
    args.gpu_per_trial = 0.2
    args.synch = True
    #exploit 
    args.quantile_fraction = 0.25
    
    #only pbt
    args.resample_probability = 0.2
    args.burn_in_period = 0
    

    args.seed = 1

    # TODO change the path as relative path
    args.to_save_path     = to_save_path
    args.freq_save_path   = freq_save_path
    args.window_save_path = window_save_path
    args.root_path        = root_path

    args.drop_transition  = False
    args.datanorm_type    = "standardization" # None ,"standardization", "minmax"

    args.batch_size       = 256                                                    
    args.shuffle          = True
    args.drop_last        = False
    args.train_vali_quote = 0.90                                           


    # training setting 
    #args.train_epochs            = 150

    args.learning_rate           = 0.001  
    args.learning_rate_patience  = 5
    args.learning_rate_factor    = 0.1


    args.early_stop_patience     = 15

    #args.use_gpu                 = True if torch.cuda.is_available() else False
    #args.gpu                     = 0
    #args.use_multi_gpu           = False

    args.optimizer               = "Adam"
    args.criterion               = "CrossEntropy"

    args.data_name                        =  "oppo"
    args.use_vali_keys = True
    

    args.wavelet_filtering                = False
    args.wavelet_filtering_regularization = False
    args.wavelet_filtering_finetuning     = False


    args.difference       = False 
    args.filtering        = False
    args.magnitude        = False
    args.weighted_sampler = False

    args.pos_select       = None
    args.sensor_select    = None


    args.representation_type = "time"
    #args.exp_mode            = "LOCV"
    args.exp_mode            = "Given"

  #  if args.data_name      ==  "skodar":
  #      args.exp_mode            = "SOCV"

    config_file = open(config_path, mode='r')
    data_config = yaml.load(config_file, Loader=yaml.FullLoader)
    config = data_config[args.data_name]

    args.root_path       = os.path.join(args.root_path,config["filename"])
    args.sampling_freq   = config["sampling_freq"]
    args.num_classes     =  config["num_classes"]
    window_seconds       = config["window_seconds"]
    args.windowsize      =   int(window_seconds * args.sampling_freq) 
    args.input_length    =  args.windowsize
    # input information
    args.c_in            = config["num_channels"]

    if args.wavelet_filtering :
        
        if args.windowsize%2==1:
            N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
        else:
            N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

        args.f_in            =  args.number_wavelet_filtering*N_ds+1
    else:
        args.f_in            =  1


    args.filter_scaling_factor = 0.25
    args.model_type            = "deepconvlstm"
    args.model_config = model_config

    #args.random_augmentation_prob = 0.5
    #args.mixup_probability  = 0.5
    #args.random_aug_first = True
    args.mixup_alpha  = 0.4
    args.mixup_argmax = True

    args.max_aug = 3

    args.random_augmentation_config = {"jitter":True,
                                    "moving_average":True,
                                    "magnitude_scaling":True,
                                    "magnitude_warp":True,
                                    "magnitude_shift":True,
                                    "time_warp":True,
                                    "window_warp":True,
                                    "window_slice":True,
                                    "random_sampling":True,
                                    "slope_adding":True
                                    }
    random_augmentation_nr = 0
    for key in args.random_augmentation_config.keys():
        if args.random_augmentation_config[key]==1:
            random_augmentation_nr = random_augmentation_nr+1
    args.random_augmentation_nr = random_augmentation_nr

    return args

In [3]:
args = get_args()
result_grid = train(args)

(train_net pid=9921) /tmp/ray/session_2023-08-16_19-05-30_904053_8929/runtime_resources/working_dir_files/_ray_pkg_f201ed243ee42fb4
(train_net pid=9921) Train data number :  47869
(train_net pid=9921) The number of classes is :  18
(train_net pid=9921) The input_length  is :  30
(train_net pid=9921) The channel_in is :  77
(train_net pid=9921) Validation data number :  2147
(train_net pid=9921) Build the DeepConvLSTM model!
(train_net pid=9924) new best score!!!!
(train_net pid=9924) /tmp/ray/session_2023-08-16_19-05-30_904053_8929/runtime_resources/working_dir_files/_ray_pkg_f201ed243ee42fb4 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(train_net pid=9924) Train data number :  47869 [repeated 4x across cluster]
(train_net pid=9924) The number of classes is :  18 [repeated 4x across cluster]
(train_net pid=992

2023-08-16 19:26:12,718	INFO pbt.py:809 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 1dddc_00004 (score = 0.846297) into trial 1dddc_00002 (score = 0.803912)

2023-08-16 19:26:12,719	INFO pbt.py:836 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of trial1dddc_00002:
random_augmentation_prob : 0.6000000000000001 --- (shift right) --> 0.7000000000000001
mixup_probability : 1.0 --- (resample) --> 0.7000000000000001

2023-08-16 19:26:12,831	INFO pbt.py:809 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 1dddc_00006 (score = 0.855147) into trial 1dddc_00011 (score = 0.816022)

2023-08-16 19:26:12,832	INFO pbt.py:836 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of trial1dddc_00011:
random_augmentation_prob : 1.0 --- (resample) --> 0.4
mixup_probability : 0.7000000000000001 --- (shift right) --> 0.8

2023-08-16 19:26:12,937	INFO pbt.py:809 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 1dddc_00003 (score = 

(train_net pid=9931) new best score!!!!


2023-08-16 19:26:13,162	INFO pbt.py:809 -- 

[PopulationBasedTraining] [Exploit] Cloning trial 1dddc_00004 (score = 0.846297) into trial 1dddc_00014 (score = 0.822543)

2023-08-16 19:26:13,163	INFO pbt.py:836 -- 

[PopulationBasedTraining] [Explore] Perturbed the hyperparameter config of trial1dddc_00014:
random_augmentation_prob : 0.6000000000000001 --- (shift left) --> 0.5
mixup_probability : 1.0 --- (shift left) --> 0.9

2023-08-16 19:26:13,268	WARNING util.py:315 -- The `scheduler.on_trial_result` operation took 0.550 s, which may be a performance bottleneck.
2023-08-16 19:26:13,269	WARNING util.py:315 -- The `process_trial_result` operation took 0.551 s, which may be a performance bottleneck.
2023-08-16 19:26:13,269	WARNING util.py:315 -- Processing trial results took 0.551 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2023-08-16 19:26:13,270	WARNING util.py:315 -- The `process_trial_result` operation took 0.552 s, which m

(train_net pid=11806) /tmp/ray/session_2023-08-16_19-05-30_904053_8929/runtime_resources/working_dir_files/_ray_pkg_f201ed243ee42fb4
(train_net pid=11806) Train data number :  47869
(train_net pid=11806) The number of classes is :  18
(train_net pid=11806) The input_length  is :  30
(train_net pid=11806) The channel_in is :  77
(train_net pid=11806) Validation data number :  2147
(train_net pid=11806) Build the DeepConvLSTM model!


...

(train_net pid=26598) /tmp/ray/session_2023-08-16_19-05-30_904053_8929/runtime_resources/working_dir_files/_ray_pkg_f201ed243ee42fb4
(train_net pid=26598) Train data number :  47869
(train_net pid=26598) The number of classes is :  18
(train_net pid=26598) The input_length  is :  30
(train_net pid=26598) The channel_in is :  77
(train_net pid=26598) Validation data number :  2147
(train_net pid=26598) Build the DeepConvLSTM model!
(train_net pid=26768) Learning rate adjusting counter: 2 out of 5
(train_net pid=26676) Learning rate adjusting counter: 2 out of 5
(train_net pid=26676) Learning rate adjusting counter: 3 out of 5


(train_net pid=26676) Restored on 10.10.253.254 from checkpoint: /home/janis/PopulationBasedTraining/RayTuneResults/oppo/pbt/oppo_test_data_as_validation_perturbation_3_1/train_net_1dddc_00010_10_mixup_probability=0.2000,random_aug_first=False,random_augmentation_prob=0.4000_2023-08-16_19-05-35/checkpoint_tmpd34363
(train_net pid=26676) Current state after restoring: {'_iteration': 21, '_timesteps_total': None, '_time_total': 2278.1859228610992, '_episodes_total': None}


(train_net pid=26676) /tmp/ray/session_2023-08-16_19-05-30_904053_8929/runtime_resources/working_dir_files/_ray_pkg_f201ed243ee42fb4
(train_net pid=26676) Train data number :  47869
(train_net pid=26676) The number of classes is :  18
(train_net pid=26676) The input_length  is :  30
(train_net pid=26676) The channel_in is :  77
(train_net pid=26676) Validation data number :  2147
(train_net pid=26676) Build the DeepConvLSTM model!
(train_net pid=26598) Learning rate adjusting counter: 1 out of 5
(train_net pid=26724) Learning rate adjusting counter: 1 out of 5
(train_net pid=23390) new best score!!!!
(train_net pid=26768) Learning rate adjusting counter: 3 out of 5


(train_net pid=26768) Restored on 10.10.253.254 from checkpoint: /home/janis/PopulationBasedTraining/RayTuneResults/oppo/pbt/oppo_test_data_as_validation_perturbation_3_1/train_net_1dddc_00011_11_mixup_probability=1.0000,random_aug_first=True,random_augmentation_prob=0.0000_2023-08-16_19-05-35/checkpoint_tmp153318
(train_net pid=26768) Current state after restoring: {'_iteration': 21, '_timesteps_total': None, '_time_total': 2278.1859228610992, '_episodes_total': None}


(train_net pid=26768) /tmp/ray/session_2023-08-16_19-05-30_904053_8929/runtime_resources/working_dir_files/_ray_pkg_f201ed243ee42fb4
(train_net pid=26768) Train data number :  47869
(train_net pid=26768) The number of classes is :  18
(train_net pid=26768) The input_length  is :  30
(train_net pid=26768) The channel_in is :  77
(train_net pid=26768) Validation data number :  2147
(train_net pid=26768) Build the DeepConvLSTM model!
(train_net pid=26676) new best score!!!!


In [ ]:
best_result = result_grid.get_best_result(mode="max", metric="mean_accuracy")
best_checkpoint = best_result.checkpoint
checkpoint_dict = best_checkpoint.to_dict()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

total_loss,  acc, f_w,  f_macro, f_micro = cal_test_accuracy(args, device, checkpoint_dict)

print(total_loss)
print(acc)